In [1]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=3, suppress=True, linewidth=100)


In [5]:
dt = 0.05
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0,0.0,0.0, 0.0])
u_vec = np.array([0.0,0.0]) 
x_seq = np.zeros((len_seq, 4))
x_seq[0]= x_init

In [6]:
mjcf_model  = mj_models.create_MJCF_double_pend_dev(1,1,1)
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)

In [7]:

nu = mj_model.nu
nx = mj_model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
mj_model.opt.timestep = dt
mj_data.ctrl = u_vec
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)

[0. 0.]
[0. 0.]


In [8]:
dpend_sys = dyn.double_pend_rel_dyn(g=9.81, m1=m_bar, moi1=moi, d1=h_bar/2, l1=h_bar, 
                                        m2=m_bar, moi2=moi, d2=h_bar/2, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=True)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [9]:
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_init, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.969  0.031  0.05   0.   ]
 [ 0.042  0.885  0.     0.05 ]
 [-0.63   0.627  1.     0.   ]
 [ 0.838 -2.3    0.     1.   ]]
A dyn: 
 [[ 0.984  0.015  0.05   0.   ]
 [ 0.021  0.943  0.     0.049]
 [-0.622  0.612  0.984  0.015]
 [ 0.817 -2.252  0.021  0.943]]
B mujoco: 
 [[ 0.004 -0.011]
 [-0.011  0.034]
 [ 0.085 -0.213]
 [-0.213  0.682]]
B dyn: 
 [[ 0.002 -0.005]
 [-0.005  0.017]
 [ 0.084 -0.209]
 [-0.209  0.668]]


In [57]:
x_vec = np.array([1.0, 1.0, 0.0, 0.0])
u_vec = np.array([0.0,0.0])
mj_funcs.set_mj_ctrl_vec(mj_data, u_vec)
mj_funcs.set_mj_state_vec(mj_data, x_vec)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_vec, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.978 -0.004  0.05   0.   ]
 [ 0.054  0.998 -0.     0.05 ]
 [-0.43  -0.086  1.     0.   ]
 [ 1.084 -0.035 -0.     1.   ]]
A dyn: 
 [[ 0.991 -0.003  0.05  -0.   ]
 [ 0.021  1.001  0.001  0.05 ]
 [-0.349 -0.122  0.98  -0.01 ]
 [ 0.841  0.019  0.061  1.012]]
B mujoco: 
 [[ 0.002 -0.004]
 [-0.004  0.015]
 [ 0.045 -0.081]
 [-0.081  0.296]]
B dyn: 
 [[ 0.001 -0.001]
 [-0.001  0.006]
 [ 0.038 -0.06 ]
 [-0.059  0.243]]


In [ ]:
pot_energy_seq = np.zeros((len_seq,1))
kin_energy_seq = np.zeros((len_seq,1))
tot_energy_seq = np.zeros((len_seq,1))
pot_energy_seq[0] = dpend_sys.calculate_potential_energy(x_init)
kin_energy_seq[0] = dpend_sys.calculate_kinetic_energy(x_init)
tot_energy_seq[0] = dpend_sys.calculate_total_energy(x_init)